In [29]:
import os
import requests
from dotenv import load_dotenv
import time

In [18]:
load_dotenv()

url = "https://ai.nutanix.com/api/v1/chat/completions" 

nai_api_key = os.getenv("NAI_API_KEY")

# print(nai_api_key)

In [31]:
question = "produce the answer. We start with 15 eggs and if 5 are taken out how many are left? think step by step "

start_time = time.time()
response = requests.post(url, 
                         headers={
                                "Authorization": f"Bearer {nai_api_key}", 
                                "accept": "application/json",
                                "Content-Type": "application/json"
                            }, 
                         json={
                                "model": "vllm-llama-3-1",
                                "messages": [
                                    {
                                        "role": "user",
                                        "content": f"{question}"
                                    }
                                ],
                                "max_tokens": 256,
                                "stream": False
                            }, 
                         verify=False)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

if response.status_code == 200:
    print("Response from the API:")
    print(response.json())
else:
    print(f"Failed with status code: {response.status_code}")
    print("Error message:", response.text)

/Users/hritik.raj/DSPy/nai-api/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Time taken: 2.651865005493164 seconds
Response from the API:
{'id': 'd2d3943d-177f-4460-8286-6eaa80970c4f', 'object': 'chat.completion', 'created': 1729682716, 'model': 'vllm-llama-3-1', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "To determine how many eggs are left after 5 are taken out, we'll follow the steps below:\n\n1. We start with 15 eggs.\n2. Next, 5 eggs are taken out. To find out how many eggs are left, we subtract 5 from 15.\n3. 15 - 5 = 10\n\nSo, there are 10 eggs left."}, 'finish_reason': 'stop', 'content_filter_results': {'hate': {'filtered': False}, 'self_harm': {'filtered': False}, 'sexual': {'filtered': False}, 'violence': {'filtered': False}, 'jailbreak': {'filtered': False, 'detected': False}, 'profanity': {'filtered': False, 'detected': False}}}], 'usage': {'prompt_tokens': 62, 'completion_tokens': 79, 'total_tokens': 141, 'prompt_tokens_details': None, 'completion_tokens_details': None}, 'system_fingerprint': ''}


In [28]:
print(response.json()["choices"][0]["message"]["content"])

To find the number of eggs left, we start with 15 and subtract the 5 eggs that were taken out.

15 (initial eggs) - 5 (eggs taken out) = 10

There are 10 eggs left.


# Step 1 : Refinning the prompt

### My primary refinning techniques would be:

- Clarity
- Context
- Conciseness

### Base Prompt?

Deciding on a base prompt is an important step, and it depends on the task we are actually trying to accomplish. Lets start with a data analysis example, assuming the client wants to generate insights from a sales report.

In [32]:
with open('prompts/prompt_01.txt', 'r') as file:
    base_prompt = file.read()

print(base_prompt)

hello


### Lets Define Rules for clarity, context and conciseness

In [35]:
generic_rules={
    "clarity":{
        "summarization":"Summarize the following text cleary and coincisely.",
        "generation":"Generate a response that clearly addresses the request.",
        "QA":"Answer the Question based on the provided information."
    },
    "context":{
        "summarization":"The text is about [subject/Topic]",
        "generation":"Make sure to include context about [subject/topic]",
        "QA":"Use the context of [subject/topic] to provide a relevant answer."
    },
    "conciseness":{
        "summarization": "Limit the summary to under [word_count] words.",
        "generation": "Keep the response concise and focused on the main idea.",
        "QA": "Provide a direct and concise answer."
    }
}

Since we have defined the rules, maybe now we will have to classify the `prompt` among these

In [33]:
def refine_clarity(prompt):
    """Classify the prompt type based on the keywords or structure"""
    if "summarize" in prompt.lower():
        return "summarization"
    elif "generate" in prompt.lower():
        return "generation"
    elif "answer" in prompt.lower():
        return "QA"
    else:
        return "generic"

In [36]:
def apply_clarity(prompt_type, prompt):
    """Apply the clarity rules based on the prompt type"""
    if prompt_type in generic_rules["clarity"]:
        return generic_rules["clarity"][prompt_type]+" "+prompt
    return prompt

def apply_context(prompt_type, prompt,subject="the provided information"):
    """Apply the context rule based on the prompt type"""
    if prompt_type in generic_rules["context"]:
        return generic_rules["context"][prompt_type].replace("[subject/topic]", subject)+" "+prompt
    return prompt

def apply_conciseness(prompt_type, prompt, word_count=100):
    """Apply consciseness rule based on prompt type"""
    if prompt_type in generic_rules["conciseness"]:
        return generic_rules["conciseness"][prompt_type].replace("[word_count]", str(word_count)) + " " + prompt
    return prompt